# Monte Carlo simulation for calculating CVA

In [2]:
# import all the necessary libraries
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import random

***Here we are capturing the information as per the problem statement. The share is the asset on
which we hold a call option, and the firm is the counterparty for the option. Barrier is limit beyond which the option expires worthless.

In [3]:
#Market information 
risk_free_rate = 0.08

#Share specific information
S_0 = 100
sigma = 0.30

#Call option specific information
strike =100
T = 1
barrier_limit=150

#Firm specific information
V_0 = 200
sigma_firm = 0.25 
debt = 175
recovery_rate = 0.25 
correlation = 0.2

***It is necessary to create the functions which transform our random standard normals into
terminal values. Note that we are going to be modeling our firm value in the same way which
we have been modeling stock values – hence, we are going to be using the terminal_value
function for both our stock and our firm. The call_payoff function finds the payoff for a call
option at terminal time.

In [4]:
#Utility Functions for later valuations
def terminal_value(S_0, risk_free_rate, sigma, Z, T):
    """Generate the terminal stock price given the random normal values, Z"""
    return S_0 * np.exp((risk_free_rate-sigma**2/2)*T+sigma*np.sqrt(T)*Z)

def call_payoff(S_T, K):
    """Function for evaluating the payoff for call option at terminal time"""
    return max(S_T-K, 0)

***Since we are working within the Merton model, we are going to be simulating our assets using a standard normal distribution.
We are using a sample size of 50 000 – this means that we have 50 000 estimates for our stock price and 50 000 estimates for our firm value. Given the correlation between shares and firm value as 0.2 we generate the correlated standard normal numbers using a Cholesky decomposition.

In [5]:
np.random.seed(0)

corr_matrix = np.array([[1, correlation], [correlation, 1]])
norm_matrix = norm.rvs(size = np.array([2, 50000]))
corr_norm_matrix = np.matmul(np.linalg.cholesky(corr_matrix), norm_matrix)

***Once we have our matrix of standard normal random numbers, we can generate our terminal stock and firm values. first create an array of stock values using the first row of the matrix of correlated standard normals. Note that this will create an array of 50 000 stock values. Then create an array of terminal firm values.

In [8]:
term_stock_val = terminal_value(S_0, risk_free_rate, sigma, corr_norm_matrix[0,],T)
call_val = term_stock_val * (term_stock_val > barrier_limit)
term_firm_val = terminal_value(V_0, risk_free_rate, sigma, corr_norm_matrix[1,],T)
print(np.mean(np.exp(-risk_free_rate * T) * call_val))

16.93150545719726


***With these terminal values generated, we can calculate our potential profit losses in each case. Losses occurs when the firm value falls below its outstanding debt, in which case we recover a percentage of outstanding which is called Recovery Rate. Finally CVA is calculated as mean of all sample generated losses.

In [7]:
amount_lost = np.exp(-risk_free_rate * T) * (1 - recovery_rate) * (term_firm_val < debt) * call_val
cva_estimate = np.mean(amount_lost)
cva_std = np.std(amount_lost)/np.sqrt(50000)
print("cva_estimate : ", cva_estimate)

#Code to calculate default probability
# d_1 = (np.log(V_0/debt) + (risk_free + sigma_firm**2/2) * (T))/(sigma_firm * np.sqrt(T))
# d_2 = d_1 - sigma_firm * np.sqrt(T)

# default_prob = norm.cdf(-d_2)

cva_estimate :  2.2580240310760042
